# Memory

## ChatMessageHistory

In [1]:
from langchain.memory import ChatMessageHistory

In [2]:
history = ChatMessageHistory()
history

InMemoryChatMessageHistory(messages=[])

In [3]:
history.add_user_message("hi!")
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [4]:
history.add_ai_message("whats up?")
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='whats up?', additional_kwargs={}, response_metadata={})])

## ConversationBufferMemory

In [5]:
from langchain.memory import ConversationBufferMemory

In [6]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("안녕")
memory.chat_memory.add_ai_message("안녕하세요 무엇을 도와드릴까요?")

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_57612/150238617.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [7]:
memory.load_memory_variables({})

{'history': 'Human: 안녕\nAI: 안녕하세요 무엇을 도와드릴까요?'}

In [8]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})]))

### memory_key
Memory에 이름을 부여

In [9]:
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("안녕")
memory.chat_memory.add_ai_message("안녕하세요 무엇을 도와드릴까요?")

In [10]:
memory.load_memory_variables({})

{'chat_history': 'Human: 안녕\nAI: 안녕하세요 무엇을 도와드릴까요?'}

Return Type을 messages로 바꿀 수도 있음

In [11]:
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("안녕")
memory.chat_memory.add_ai_message("안녕하세요 무엇을 도와드릴까요?")

In [12]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})]}

## ConversationBufferWindowMemory

In [13]:
from langchain.memory import ConversationBufferWindowMemory

In [14]:
memory = ConversationBufferWindowMemory(k=1, return_messages=True)

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_57612/3925694220.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1, return_messages=True)


In [15]:
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory

ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='whats up', additional_kwargs={}, response_metadata={})]), return_messages=True, k=1)

In [16]:
memory.save_context({"input": "not much you"}, {"output": "not much"})
memory

ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='whats up', additional_kwargs={}, response_metadata={}), HumanMessage(content='not much you', additional_kwargs={}, response_metadata={}), AIMessage(content='not much', additional_kwargs={}, response_metadata={})]), return_messages=True, k=1)

In [17]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='not much you', additional_kwargs={}, response_metadata={}),
  AIMessage(content='not much', additional_kwargs={}, response_metadata={})]}

# LangChain으로 대화 관리하기

In [18]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [19]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "너는 친절한 어시스턴트야."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{user_text}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_57612/819824388.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


In [20]:
conversation.invoke({"user_text": "안녕"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 친절한 어시스턴트야.
Human: 안녕

> Finished chain.


{'user_text': '안녕',
 'chat_history': [HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?👋', additional_kwargs={}, response_metadata={})],
 'text': '안녕하세요! 무엇을 도와드릴까요?👋'}

In [21]:
conversation.invoke({"user_text": "배고파"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 친절한 어시스턴트야.
Human: 안녕
AI: 안녕하세요! 무엇을 도와드릴까요?👋
Human: 배고파

> Finished chain.


{'user_text': '배고파',
 'chat_history': [HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?👋', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='배고파', additional_kwargs={}, response_metadata={}),
  AIMessage(content='배고프시다면 무엇을 먹을지 고민 중이신가요? 영양가 있는 식사를 즐기는 건 중요하니, 드시고 싶은 음식이 있으면 추천해 드릴게요! 🍽️✨', additional_kwargs={}, response_metadata={})],
 'text': '배고프시다면 무엇을 먹을지 고민 중이신가요? 영양가 있는 식사를 즐기는 건 중요하니, 드시고 싶은 음식이 있으면 추천해 드릴게요! 🍽️✨'}